In [1]:
pip install line-bot-sdk

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [9]:
from flask import Flask, request, abort
from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,    
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
)
import pandas as pd
import os
from linebot.v3.messaging import (
    FlexMessage, MessageAction
)

# 從環境變數中獲取 Token 和 Secret
line_access_token = os.environ.get('LINE_ACCESS_TOKEN')
line_secret = os.environ.get('LINE_SECRET')

# 初始化 Flask
app = Flask(__name__)

configuration = Configuration(access_token=line_access_token)
handler = WebhookHandler(line_secret)
hotel_data = pd.read_csv('./Hotel_E_F.csv')
food_data = pd.read_csv('./Taiwan_Food_5000.csv')
attraction_data = pd.read_csv('./Taiwan_Attractions_5000.csv')
# 使用者選擇的城市和地區紀錄
user_city = {}
user_state = {}
ITEMS_PER_PAGE = 10
cities = {
    "台北": "63",
    "新北": "65",
    "基隆": "10017",
    "桃園": "68",
    "新竹市": "10018",
    "新竹縣": "10004",
    "苗栗": "10005",
    "台中": "66",
    "彰化": "10007",
    "南投": "10008",
    "雲林": "10009",
    "嘉義市": "10020",
    "嘉義縣": "10010",
    "台南": "67",
    "高雄": "64",
    "屏東": "10013",
    "宜蘭": "10002",
    "花蓮": "10015",
    "台東": "10014",
    "澎湖": "10016",
    "金門": "09020",
    "連江": "09007"
}

def get_hotel_flex_message(hotels):
    """
    使用 Flex Message 的方式回傳 Template
    """
    columns = []
    for index, row in hotels.iterrows():
        bubble = {
            "type": "bubble",
            "hero": {
                "type": "image",
                "url": "https://example.com/hotel.jpg",
                "size": "full",
                "aspectRatio": "20:13",
                "aspectMode": "cover"
            },
            "body": {
                "type": "box",
                "layout": "vertical",
                "contents": [
                    {
                        "type": "text",
                        "text": row['旅館名稱'],
                        "weight": "bold",
                        "size": "lg",
                        "margin": "md"
                    },
                    {
                        "type": "text",
                        "text": f"💰 {row['最低房價']} ~ {row['最高房價']} 元",
                        "size": "sm",
                        "color": "#555555",
                        "wrap": True
                    },
                    {
                        "type": "text",
                        "text": f"📍 {row['地址']}",
                        "size": "sm",
                        "color": "#555555",
                        "wrap": True
                    },
                    {
                        "type": "button",
                        "action": {
                            "type": "message",
                            "label": "📞 打電話",
                            "text": f"撥打 {row['電話']}"
                        },
                        "style": "primary",
                        "color": "#1DB446",
                        "margin": "md"
                    }
                ]
            }
        }
        columns.append(bubble)
        
        # LINE Flex Message Carousel 最多 10 筆
        if len(columns) == 10:
            break

    # 使用 Carousel Template 來顯示
    if len(columns) > 0:
        flex_message = FlexMessage(
            alt_text="住宿資訊",
            contents={
                "type": "carousel",
                "contents": columns
            }
        )
    else:
        flex_message = None

    return flex_message

@app.route("/", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)
    except Exception as e:
        print(f"Error: {e}")
        abort(400)
        
    return 'OK'

# 處理文字訊息
@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    user_message = event.message.text.strip()
    user_id = event.source.user_id

    # 每次事件都初始化 API Client
    with ApiClient(Configuration(access_token=line_access_token)) as api_client:
        line_bot_api = MessagingApi(api_client)
        if user_message == "選擇城市":
            reply_text = (
                "請選擇想去的縣市：\n台北、新北、基隆、桃園、新竹市、新竹縣、苗栗、台中、彰化、南投、雲林、嘉義市、嘉義縣、台南、高雄、屏東、宜蘭、花蓮、台東、澎湖、金門、連江\n"
                "格式：<城市> <地區>\n例如：台北 士林區 或 桃園 中壢區"
            )
        elif " " in user_message:
            city, district = user_message.split(" ", 1)
            if city in cities:
                user_city[user_id] = {"city": city, "district": district}
                reply_text = f"您已選擇城市：{city}，地區：{district}\n接下來可以輸入「景點推薦」、「美食推薦」、「住宿資訊」或「即時資訊」來查詢資料。"
            else:
                reply_text = "無效的城市名稱，請重新輸入。"    
        elif user_message.startswith("住宿資訊"):
            if user_id in user_city:
                city_info = user_city[user_id]
        
                city_hotels = hotel_data[(hotel_data['城市'] == city_info['district'])]

                if not city_hotels.empty:
                    flex_message = get_hotel_flex_message(city_hotels)

                    # 回覆訊息
                    if flex_message:
                        reply_text = flex_message
                    else:
                        reply_text = "找不到相關的住宿資訊。"
            
                else:
                    reply_text = f"找不到 {city_info['city']} {city_info['district']} 的住宿資訊。"
            else:
                reply_text = "請先選擇一個城市和地區，再輸入「住宿資訊」。"    
        
        
        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=reply_text)]
            )
        )

if __name__ == "__main__":
    app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [19/May/2025 22:40:07] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2025 22:40:13] "POST / HTTP/1.1" 400 -


Error: cannot access local variable 'reply_text' where it is not associated with a value


127.0.0.1 - - [19/May/2025 22:40:16] "POST / HTTP/1.1" 400 -


Error: 1 validation error for TextMessage
text
  str type expected (type=type_error.str)


...

Error: cannot access local variable 'reply_text' where it is not associated with a value


127.0.0.1 - - [19/May/2025 22:40:52] "POST / HTTP/1.1" 400 -


Error: cannot access local variable 'reply_text' where it is not associated with a value


127.0.0.1 - - [19/May/2025 22:40:55] "POST / HTTP/1.1" 400 -


Error: cannot access local variable 'reply_text' where it is not associated with a value
